In [3]:
!pip install transformers
!pip install torch

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load your model and tokenizer (do this once when the module is imported)
model_path = "./fine_tuned_llama"

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
print("model and tokenizer loaded")
def generate_sql(instruction, context):
    # prompt = f"### Instruction: {instruction}\n\n### Context: {context}"
    prompt = f"### Instruction: {instruction}\n\n### Context: {context}\n\n### Response:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=1000,
            temperature=0.2,
            # max_time=120
        )
    ans= tokenizer.decode(outputs[0], skip_special_tokens=True)
    del tokenizer
    return ans

instruction = """Which habitats have seen a decrease in preservation efforts from 2017 to 2018?"""
context="""CREATE TABLE PreservationTrends(Year INT, Habitat VARCHAR(20), Efforts INT); INSERT INTO PreservationTrends VALUES (2017, 'Forest', 120), (2018, 'Forest', 150), (2017, 'Wetland', 80), (2018, 'Wetland', 90);"""

print(generate_sql(instruction,context))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


model and tokenizer loaded
### Instruction: Which habitats have seen a decrease in preservation efforts from 2017 to 2018?

### Context: CREATE TABLE PreservationTrends(Year INT, Habitat VARCHAR(20), Efforts INT); INSERT INTO PreservationTrends VALUES (2017, 'Forest', 120), (2018, 'Forest', 150), (2017, 'Wetland', 80), (2018, 'Wetland', 90);

### Response: SELECT Habitat, Year, Efforts FROM PreservationTrends WHERE Efforts < (SELECT SUM(Efforts) FROM PreservationTrends WHERE Year = 2018);

### Explanation: This query selects the Habitat, Year, and Efforts columns from the PreservationTrends table where the Efforts column is less than the sum of Efforts from the PreservationTrends table where Year = 2018. This gives us the Habitat, Year, and Efforts for habitats that have seen a decrease in preservation efforts from 2017 to 2018.
